# **Installing all the defendecy**

In [1]:
!pip install tensorflow-gpu==2.0.0.alpha0

     |████████████████████████████████| 332.5MB 33kB/s 
     |████████████████████████████████| 3.0MB 27.3MB/s 
     |████████████████████████████████| 419kB 39.2MB/s 
     |████████████████████████████████| 51kB 5.9MB/s 


In [2]:
!pip install tqdm


# DOWNLOADING THE DATASET

In [3]:
!!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip


['--2021-04-05 11:04:09--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip',
 'Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.76.128, 66.102.1.128, 74.125.206.128, ...',
 'Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.76.128|:443... connected.',
 'HTTP request sent, awaiting response... 200 OK',
 'Length: 68606236 (65M) [application/zip]',
 'Saving to: ‘./cats_and_dogs_filtered.zip’',
 '',
 '',
 '          ./cats_an   0%[                    ]       0  --.-KB/s               ',
 '         ./cats_and  14%[=>                  ]   9.58M  42.8MB/s               ',
 '        ./cats_and_  27%[====>               ]  18.15M  34.6MB/s               ',
 '       ./cats_and_d  51%[=========>          ]  34.01M  42.2MB/s               ',
 './cats_and_dogs_fil 100%[===================>]  65.43M  65.2MB/s    in 1.0s    ',
 '',
 '2021-04-05 11:04:11 (65.2 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]',
 '']

In [4]:
!pip uninstall -y tensorflow-gpu


Uninstalling tensorflow-gpu-2.0.0a0:
  Successfully uninstalled tensorflow-gpu-2.0.0a0


In [5]:
!pip install tensorflow-gpu

     |████████████████████████████████| 394.3MB 38kB/s 


# Import Dependencies 





*  zipfile is for the zip folder
*  imageDatagenerator is for automatic preprocessing 


In [6]:
import os 
import zipfile
import numpy as np 

import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator# for automatic preprocessing


/usr/local/lib/python3.7/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

# Addding the path and unziping the folder

#### mode r mean for reading purpose unzip the file 
#### ./ mean to execute the  current directory
mode="r" mean uzip the folder for reading


In [7]:
dataset_path = "./cats_and_dogs_filtered.zip"
zip_object=zipfile.ZipFile(file=dataset_path , mode = "r")
zip_object.extractall("./")
zip_object.close()

### Adding the Path of New Directory of Dataset which is downloded recently 

In [8]:
dataset_path_new = "./cats_and_dogs_filtered/"

### adding the path to training and testing
### these paths are given from the above directory


In [9]:
train_dir = os.path.join(dataset_path_new, "train") # For Training 
validation_dir=os.path.join(dataset_path_new ,"validation") # For Testing  

### The Model need a  specific input shape so we will do for it 

In [10]:
IMG_SHAPE = (128 ,128 ,3)

## Load the model and assign it to a variable 

In [11]:
base_model = tf.keras.applications.MobileNet(input_shape=IMG_SHAPE , include_top=False , weights = "imagenet")

17227776/17225924 [==============================] - 1s 0us/step


### Check the sumarray of the Model


In [12]:
base_model.summary()

Model: "mobilenet_1.00_128"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 64, 64, 32)        864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 64, 64, 32)        128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 64, 64, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 64, 64, 32)        288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 64, 64, 32)        128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 64, 64, 32) 

### Freezing the model 
#### The aim of freezing the model is not change the weights 
##### We will train the model and add some layer for our custom requrirments

In [13]:
base_model.trainable = False

### after freezing the model we are going to add customs layers  so for that we need to know the output of the model first 

In [14]:
base_model.output

<KerasTensor: shape=(None, 4, 4, 1024) dtype=float32 (created by layer 'conv_pw_13_relu')>

### So the ouput of the model is not of our need so we will do add  <mark>Global Average pooling pooling layer<mark>
which works like maxpooling layer 
it will take the whole input and find out the average of that 


In [15]:
global_average_layer= tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

### Now Add your prediction layer 

In [16]:
prediction_layer = tf.keras.layers.Dense(units=1 , activation = 'sigmoid' ) (global_average_layer)

# Defining model

In [18]:
model = tf.keras.models.Model(inputs=base_model.input , outputs=prediction_layer)

# Compiling the model

In [20]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

# Creating Data Generators

In [23]:
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

In [24]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 2000 images belonging to 2 classes.


In [25]:
valid_generator = data_gen_valid.flow_from_directory(validation_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 1000 images belonging to 2 classes.


 # fitting the mode 

In [26]:
model.fit_generator(train_generator, epochs=5, validation_data=valid_generator)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
16/16 [==============================] - 41s 667ms/step - loss: 0.7821 - accuracy: 0.5349 - val_loss: 0.6962 - val_accuracy: 0.6080
Epoch 2/5
16/16 [==============================] - 9s 574ms/step - loss: 0.6635 - accuracy: 0.6237 - val_loss: 0.6159 - val_accuracy: 0.6700
Epoch 3/5
16/16 [==============================] - 9s 570ms/step - loss: 0.6112 - accuracy: 0.6758 - val_loss: 0.5485 - val_accuracy: 0.7190
Epoch 4/5
16/16 [==============================] - 9s 564ms/step - loss: 0.5409 - accuracy: 0.7113 - val_loss: 0.4889 - val_accuracy: 0.7600
Epoch 5/5
16/16 [==============================] - 9s 570ms/step - loss: 0.4734 - accuracy: 0.7683 - val_loss: 0.4370 - val_accuracy: 0.7930


# Model Evaluation

In [27]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


In [28]:
print("Accuracy after transfer learning: {}".format(valid_accuracy))

Accuracy after transfer learning: 0.7929999828338623


# <mark>Fine Tuning <mark>

# UnFreez The Model 

In [29]:
base_model.trainable = True

# Showing How much layers 

In [31]:
print("Number of layers in the base model: {}".format(len(base_model.layers)))

Number of layers in the base model: 86


# Adding How much layers are fine tune

In [32]:
fine_tune_at = 40

# unfreezing the layers which are specified and freeze the rest 

In [33]:
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Compiling the model 

In [ ]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# fItting the model 

In [34]:
model.fit_generator(train_generator,  
                    epochs=5, 
                    validation_data=valid_generator)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
16/16 [==============================] - 10s 625ms/step - loss: 0.4113 - accuracy: 0.8200 - val_loss: 0.3936 - val_accuracy: 0.8230
Epoch 2/5
16/16 [==============================] - 9s 594ms/step - loss: 0.3696 - accuracy: 0.8470 - val_loss: 0.3560 - val_accuracy: 0.8460
Epoch 3/5
16/16 [==============================] - 9s 601ms/step - loss: 0.3341 - accuracy: 0.8660 - val_loss: 0.3235 - val_accuracy: 0.8630
Epoch 4/5
16/16 [==============================] - 9s 601ms/step - loss: 0.3030 - accuracy: 0.8870 - val_loss: 0.2951 - val_accuracy: 0.8810
Epoch 5/5
16/16 [==============================] - 9s 596ms/step - loss: 0.2765 - accuracy: 0.8990 - val_loss: 0.2703 - val_accuracy: 0.9010


# Evaluate 

In [35]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


In [36]:
print("Validation accuracy after fine tuning: {}".format(valid_accuracy))

Validation accuracy after fine tuning: 0.9010000228881836


In my case the accuracy become improved after fine tuning but there are some casas in which accuracy is not improving the accuracy can be decrease by fine tunning 